In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt 
%matplotlib qt
from findiff import FinDiff
from scipy.optimize import fsolve
import sys
sys.path.insert(0,'.')
import pandas as pd

In [6]:
# Fixing all the constants of the Problem
M = 1
n = 2**14         # no of grid points
h = 1/n       # Step size in z (our compact variable)
a = 0.1           # Kreiss Oliger Dissipation coefficient

# Parameters of the potential
mu = -200
lam = 0.1
c = 1000      # constant value given to the field

In [7]:

# We will try to solve for the function zeta using explicit method at the intital time
# Kriess Oliger Dissipation function has also been employed

def zeta_zero(p_given):

    p = p_given
    
    err = 10**(-3)                                            # Putting a cutoff to our space 
    Z = np.linspace(err, 1 - err, n)
    R = M * Z / (1 - Z)
                        
    # Specifying intitial values of the Field namely psi and pi
    psi = np.zeros(n) 

    for i in range(0, n):
        expo = Z[i]**2 / ( p**2 * (1 - Z[i])**2 )
        if expo <= 100 :
            psi[i] =  0 * np.exp(-expo)

    # Specifying hte initital value of pi 
    pi = np.zeros(n)
    
    # Defining the derivative operator using FinDiff
    dz = Z[1] - Z[0]
    d_dz = FinDiff(0, dz, 1, acc=4)

    dphi = d_dz(psi) / d_dz(R)
    d2phi = d_dz(dphi) / d_dz(R)
    dpi = d_dz(pi) / d_dz(R)
    drdz = d_dz(R)

    # Defining the potential function
    def V(x):
        return mu * x ** 2 + lam * x ** 4 

    V_phi = V(c + psi)

    # Defining the RHS of our differenetial equation 

    def term(x, i):
        term1 = (R[i] ** 2 * V_phi[i] * ((c + psi[i]) + R[i] * dphi[i]) + 2* (3 * (R[i] * dphi[i]) ** 3 + (R[i] * pi[i]) ** 2 * (pi[i] - 3 * R[i] * dphi[i]) + \
                 np.sqrt(R[i]) * pi[i] * (c + psi[i]) * (x * (3 * (c + psi[i]) + 8 * R[i] * dphi[i] ) - 2 * R[i] ** (5/2) * dpi[i] ) + R[i] ** 2 * (c + psi[i]) * dphi[i] * ( \
                 7 * dphi[i] + 2 * R[i] * d2phi[i] ) + (c + psi[i]) ** 2 * (4 * R[i] * dphi[i] - x ** 2 * dphi[i] + 2 * R[i] ** (3/2) * x * dpi[i] +  2 * R[i] **2 * d2phi[i] )))
             
        term2 = 4 * (c + psi[i]) ** 2 * (- R[i] ** (3/2) * pi[i] + x * (c + psi[i])) 

        return drdz[i] * (term1 / term2)

    # Defining the four element array for initial Euler method
    S_ini = np.zeros(4)
    S_ini[3] = np.sqrt(2 * M) # Boundary condition

    # Performing finite difference using first order difference (Eulers Method)

    for i in range(0, 3):
        S_ini[2 - i] = S_ini[3 - i] - dz * term(S_ini[3 - i] , n - i - 1) 

        # print(S_ini[2 - i])
    
    # Using fourth order finite difference with Kliess Oliger dissipation
    S = np.zeros(n)

    # Initial condition from Eulers Method
    S[n - 1] = S_ini[3]
    S[n - 2] = S_ini[2]
    S[n - 3] = S_ini[1]
    S[n - 4] = S_ini[0]

    for i in range(0, n - 4):
        def func(x):
            return  - (25 + 35 * a * dz) * x + (48 - 104 * a * dz) * S[n - i - 4] - (36 - 114 * a * dz) * S[n - i - 3] +  (16 - 56 * a * dz) * S[n - i - 2] \
                    - (3 - 11 * a * dz) * S[n - i - 1] - 12 * dz * term(x , n - i - 5)

        S[n - i - 5] = fsolve(func, x0 = S[n - i - 4])[0]
        # print(S[n - i - 5])
        
    cut_off = np.argmin(abs(S - np.sqrt(R)))

    print(f" The position of initial apparant horizon is r = {R[cut_off]} \n z = {Z[cut_off]} \n m = {cut_off}") 
    
    """
    m = n - cut_off + 1
    s = S[cut_off:]
    r = R[cut_off:]
    z = Z[cut_off:]
    
    """
    return  cut_off, Z, R, S

start_time = time.time()
data = zeta_zero(1)
end_time = time.time()
print(end_time-start_time)

plt.plot(data[1], data[3])
plt.axhline(y=1, color='r')
plt.show()

c:\Users\suriy\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


 The position of initial apparant horizon is r = 998.9999999999991 
 z = 0.999 
 m = 16383
4.952509880065918


In [37]:

plt.plot(data[1][data[0]:], data[3][data[0]:], '.')
plt.show()
print(max(data[3]))

2704262941.6542635
